## Import libraries

In [1]:
import logging
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
logging.getLogger('tensorflow').setLevel(logging.ERROR)

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import random as rd
#from keras.utils.np_utils import to_categorical
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,roc_curve,auc
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPool2D,BatchNormalization
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
import itertools
import warnings
warnings.filterwarnings("ignore")

In [4]:
from google.colab import drive

drive.mount('/content/drive')


!cp /content/drive/MyDrive/Abdulla_OCR/Character/anotation_28x28/Train.csv .
!cp /content/drive/MyDrive/Abdulla_OCR/Character/anotation_28x28/Test.csv .

!rm -rf sample_data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data load and review

In [5]:
train=pd.read_csv("Train.csv")
test=pd.read_csv("Test.csv")
# sample_sub=pd.read_csv("sample_submission.csv")

In [6]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
x_train=train.drop(columns="label",axis=1)
y_train=train["label"]

## Visualization

In [8]:
fig=px.histogram(x=y_train,text_auto=True,color_discrete_sequence=['royalblue'])
fig.update_layout(bargap=.1,
                  xaxis=dict(type="category",categoryorder="total descending"),
                  title_font_color="darkblue",
                  title="Number of images",
                  xaxis_title="Image labels",
                  height=480,
                  width=700)
fig.update_traces()
fig.show()

In [9]:
img=[]
for i in range(max(y_train.value_counts().index)+1):   
    index=rd.choice(y_train[y_train.values==i].index.values)
    image=x_train.iloc[index].to_numpy().reshape((28,28))
    img.append(image)
fig=px.imshow(np.array(img),facet_col=0, binary_string=True, facet_col_wrap=5, labels={'facet_col':'Number'})
fig.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)
fig.update_layout(title="Random picture of each number",
                 height=500,
                 width=800)
fig.show()

## Preprocessing

In [10]:
X_train=x_train/255.0
Test=test/255.0
print("x train shape: ",X_train.shape)
print("test shape: ",Test.shape)

x train shape:  (209966, 784)
test shape:  (34966, 784)


In [11]:
X_train=x_train.values.reshape(-1,28,28,1)
Test=test.values.reshape(-1,28,28,1)
print("x train shape: ",X_train.shape)
print("test shape: ",Test.shape)

x train shape:  (209966, 28, 28, 1)
test shape:  (34966, 28, 28, 1)


In [12]:
Y_train=to_categorical(y_train,num_classes=35)

In [13]:
X_train,X_val,Y_train,Y_val=train_test_split(X_train,Y_train,test_size=0.2,random_state=42)
print(f'Shape of training data: {X_train.shape}')
print(f'Shape training labels: {Y_train.shape}')
print(f'Shape of validation data: {X_val.shape}')
print(f'Shape of valiation labels: {Y_val.shape}')
print(f'Shape of testing data: {test.shape}')

Shape of training data: (167972, 28, 28, 1)
Shape training labels: (167972, 35)
Shape of validation data: (41994, 28, 28, 1)
Shape of valiation labels: (41994, 35)
Shape of testing data: (34966, 784)


In [14]:
epochs=100
BATCH_SIZE = 32000 # number of sample

In [15]:
BUFFER_SIZE = 10000
BATCH_SIZE_PER_REPLICA = 1024

def get_batch_size(num_gpus=1):
    batch_size = BATCH_SIZE_PER_REPLICA * num_gpus
    print(f'BATCH_SIZE = {batch_size}')
    return batch_size

In [16]:
datagen=ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=30,
        zoom_range=0.2,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=False,
        vertical_flip=False
)        

In [17]:
train_generator = datagen.flow(
    X_train, Y_train, batch_size=BATCH_SIZE)

In [18]:
val_generator = datagen.flow(
    X_val,Y_val, batch_size=BATCH_SIZE)

## Make data compatible with GPU

In [19]:
train_data = tf.data.Dataset.from_tensor_slices((train_generator[0][0], train_generator[0][1]))
#test_data = tf.data.Dataset.from_tensor_slices((val_generator[0][0], val_generator[0][1]))

In [20]:
#train_data = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
test_data = tf.data.Dataset.from_tensor_slices((X_val, Y_val))

In [21]:
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.OFF
train_data = train_data.with_options(options).cache().shuffle(BUFFER_SIZE).prefetch(tf.data.AUTOTUNE)
test_data = test_data.with_options(options)

In [22]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 1


## Create model

In [23]:
optimizer=Adam(learning_rate=0.001,beta_1=0.9,beta_2=0.999)
# optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

def create_model():
    model=Sequential()
    model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)))
    model.add(BatchNormalization())

    model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu'))
    model.add(BatchNormalization())

    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
    model.add(BatchNormalization())

    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same',  activation ='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256, activation = "relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    model.add(Dense(35, activation = "softmax"))
    return model
   



In [24]:
#model.compile(optimizer=optimizer,loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['sparse_categorical_accuracy'],steps_per_execution = 50)
with strategy.scope():
    model=create_model()
    model.compile(optimizer=optimizer,loss="categorical_crossentropy",metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        1664      
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        102464    
                                                                 
 batch_normalization_1 (Batc  (None, 28, 28, 64)       256       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                        

## Train Model

In [25]:
d_train_data = train_data.batch(get_batch_size(strategy.num_replicas_in_sync))
d_test_data = test_data.batch(get_batch_size(strategy.num_replicas_in_sync))
history = model.fit(d_train_data, epochs=epochs,validation_data=d_test_data)

BATCH_SIZE = 1024
BATCH_SIZE = 1024
Epoch 1/100
32/32 [==============================] - 25s 260ms/step - loss: 3.4631 - accuracy: 0.1261 - val_loss: 25.6642 - val_accuracy: 0.0687
Epoch 2/100
32/32 [==============================] - 6s 197ms/step - loss: 2.1720 - accuracy: 0.3570 - val_loss: 2.5661 - val_accuracy: 0.3117
Epoch 3/100
32/32 [==============================] - 7s 213ms/step - loss: 1.4498 - accuracy: 0.5435 - val_loss: 1.0332 - val_accuracy: 0.6663
Epoch 4/100
32/32 [==============================] - 6s 198ms/step - loss: 1.0588 - accuracy: 0.6659 - val_loss: 0.6748 - val_accuracy: 0.7810
Epoch 5/100
32/32 [==============================] - 7s 210ms/step - loss: 0.8235 - accuracy: 0.7369 - val_loss: 0.5766 - val_accuracy: 0.8019
Epoch 6/100
32/32 [==============================] - 7s 213ms/step - loss: 0.6797 - accuracy: 0.7849 - val_loss: 0.4623 - val_accuracy: 0.8484
Epoch 7/100
32/32 [==============================] - 7s 225ms/step - loss: 0.5675 - accuracy: 0.8213 - v

## Evaluation

In [26]:
loss=pd.DataFrame({"Loss":history.history["loss"],"Val_loss":history.history['val_loss']})
fig = px.line(loss)
fig.update_layout(
                 title="Loss",
                 height=300,
                 width=500,
                 xaxis=dict(title="Epoch"),
                 yaxis=dict(title="Rate"),
                 legend_title="")
fig.show()

accuracy=pd.DataFrame({"Accuracy":history.history['accuracy'],"Val_accuracy":history.history['val_accuracy']})
fig = px.line(accuracy)
fig.update_layout(
                 title="Accuracy",
                 height=300,
                 width=500,
                 xaxis=dict(title="Epoch"),
                 yaxis=dict(title="Rate"),
                 legend_title="")
fig.show()

In [27]:
from keras.models import load_model

model = model.save('model.h5')